In [6]:
import os
import csv
from bing_image_downloader import downloader

# Define classes and labels
classes = {
    'Myanmar landmarks': 'landmark',
    'Myanmar traditional clothing': 'clothing',
    'Myanmar traditional sports': 'sports'
}

# Directory for saving images
output_dir = 'dataset'

# creating the output dir and csv file
csv_file = os.path.join(output_dir, 'labels.csv')
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['filename', 'label'])

    # Download images and write labels
    for label, class_label in classes.items():
        downloader.download(label, limit=10, output_dir=output_dir, adult_filter_off=True, force_replace=False, timeout=60, verbose=True)

        # Look for the directory if it was created or not
        class_dir = os.path.join(output_dir, label)

        # Check if the directory exists and write the file name
        if os.path.isdir(class_dir):
            for img in os.listdir(class_dir):
                writer.writerow([os.path.join(class_dir, img), class_label])
        else:
            print(f"Directory for {label} not found.")


[%] Downloading Images to /Users/rei/Dev/ml/lulab/dataset/Myanmar landmarks


[!!]Indexing page: 1

[%] Indexed 40 Images on Page 1.


[%] Downloading Image #1 from https://www.tripsavvy.com/thmb/If3SV1fJ4vBZNQiv75JojvdVWYI=/4284x2856/filters:no_upscale():max_bytes(150000):strip_icc()/shwedagon-pagoda-at-sunset--yagon--myanmar-977553146-5bbea663c9e77c0051d23339.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://thetravelscribes.com/wp-content/uploads/2020/03/Landmarks-in-Myanmar-Uppatasanti-pagoda-naypyitaw-exterior-1080x811.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.exoticvoyages.com/uploads/images/userfiles/2016/12/Shwedagon-Pagoda-Attractions-in-Myanmar.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://d1bv4heaa2n05k.cloudfront.net/other/59df4c1e-45e8-4ea8-a8f7-1db3834d66c2_1533034363905/1533034623977-shutterstock_554325643.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://res.cloudinary.com/enchanting/q_70,f_auto

## Organizing data to upload to Hugging Face

In [1]:
import os
import pandas as pd
from datasets import Dataset, Features, ClassLabel, Value, Image
from datasets import load_dataset
from huggingface_hub import HfApi, HfFolder

In [48]:
img_dir = '/Users/rei/Dev/ml/lulab/mm_multi_class'

# Load the CSV file
csv_path = os.path.join(img_dir, 'dataset/labels.csv')
df = pd.read_csv(csv_path)

In [49]:
print(df.columns)

Index(['filename', 'label'], dtype='object')


In [50]:
df = df.rename(columns={'filename': 'image'})

In [51]:
df['image'] = df['image'].apply(lambda x: os.path.join(img_dir, x) if not x.startswith(img_dir) else x)
# df['label'] = df['label'].astype(str)

In [52]:
features = Features({
    'image': Image(),
    'label': ClassLabel(names=df['label'].unique().tolist())
})

In [53]:
dataset = Dataset.from_pandas(df[['image','label']], features=features)

In [54]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 30
})

In [55]:
dataset.push_to_hub("AggaMin/mm_mutli_class")

Uploading the dataset shards: 100%|██████████| 1/1 [00:10<00:00, 10.65s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/AggaMin/mm_mutli_class/commit/22ad36a43842acf58b0e92e270214d99df5bb0db', commit_message='Upload dataset', commit_description='', oid='22ad36a43842acf58b0e92e270214d99df5bb0db', pr_url=None, pr_revision=None, pr_num=None)